In [13]:
memory_size = 71
max_coordinate = memory_size-1
find_path = []
fall_bytes_number = 2930
max_fall_bytes = 0
fall_bytes = []


file_name = 'input_'+str(max_coordinate)+'.txt'
file_lines = open(file_name, 'r').read().split()
for line in file_lines:
     fall_bytes.append(list(map(int,line.split(','))))

def cut_fall_bytes_list (list, max) :
    for i in range (len(list)-max) :
        list.pop()
    return list

max_fall_bytes =  len(fall_bytes)
all_fall_bytes = list(fall_bytes)
fall_bytes = cut_fall_bytes_list(fall_bytes, fall_bytes_number)


def print_map (path=[]) :
    print("Memory with fall bytes :")
    space_btw = ""
    for y in range (memory_size) :
        line = space_btw
        for x in range (memory_size) :            
            if [x,y] in fall_bytes :
                line += "#"
            else :
                if [x,y] in path :
                    line += "o"
                else :
                    line += "."
        line += space_btw
        print(line)
    return

print_map()

Memory with fall bytes :
..#..#.##..#...........#...#...#.....#.....#.##..#...##..#.....#......#
.###.###.###.#########.###.#.#.#.###.###.#.#####.#.#.#.###.###########.
.#.#.#...#.#.#..##...#...#.#.#..##.#.#..##...#.###.#.#.#.##....#...#...
.#.###.###.#.###.#######.#########.###########.#.#.#.#.#########.######
.#.....#.#.#...#.#.#.#.#.#..#..###.###..#..#.#.#.#.#.#.####..#.#....#..
.#######.###.###.#.#######.###.#####.#####.###.#.#.#.#######.#########.
.......###.#.#...#.....#.###...##..#.....#.#.#...#.##..#...#..##.#.....
.#####.#.#.#.#.###.#######.###############.#.#####.###.#.#########.###.
.#..##.#...#.#...##....#.#.#.......#.....#.#...#.#.#.#...#...#...#.#.#.
.#####.#######.#.#####.#.#########.#.#####.#####.#.#######.#########.#.
.....#.#.#.#.###.###..##.##..#.#.#.#...#.#.#.#.#.###.#.#####..#..#.#...
######.#####.#####.#.#####.#.###.#####.#.###.#.###.#.#.###########.#.##
.....#.###.#..#.##.#.##..#.##....#.#...##.##.....#...#.#.#.#.#.....#.#.
.###.#.#.#########.###.###.###.###.####

In [14]:
border_top = 0
border_bottom = max_coordinate
border_left = 0
border_right = max_coordinate

def give_moves_data(x,y) :
    moves = [
        "top",
        "bottom",
        "left",
        "right"
    ]
    borders = {
        moves[0] : y!=border_top,
        moves[1] : y!=border_bottom,  
        moves[2] : x!=border_left, 
        moves[3] : x!=border_right
    }
    moves_coordinates = {
        moves[0] : [x,y-1],
        moves[1] : [x,y+1],
        moves[2] : [x-1,y],
        moves[3] : [x+1,y]
    }
    return moves, borders, moves_coordinates

In [15]:
def discover_path (x,y, path):
    #print(".",end="")
    #if steps == 5 :
    #    print("Searching path for "+str(x)+","+str(y))
    moves, borders, moves_coordinates = give_moves_data(x,y)
    can_move = {}
    for i in range (len(moves)) :
        can_move[moves[i]] = (borders[moves[i]] 
                              and not(moves_coordinates[moves[i]] in fall_bytes)
                              and not(moves_coordinates[moves[i]] in path))
    
    move_find = False
    for move in moves :
        #if steps == 5 :
        #    print("Try",move)
        if can_move[move] :
                if not(moves_coordinates[move] in reached_point) : # Give up path which reach a coordinates allready reach by another faster
                    reached_point.append(moves_coordinates[move]) 
                    check_reach_goal(moves_coordinates[move],path)
                    add_path(path, moves_coordinates[move])     
                    move_find = True
    return

def add_path (root_path, coordinates) :
    global reached_point
    global new_paths
    #if  goal_reached:
    #    print("Adding to path",root_path,"coordinates",coordinates)
    path = list(root_path)
    path.append(list(coordinates))
    new_paths.append(list(path) )
    #print("Root path",path)
    #print("New paths",new_paths)      
    return

def check_reach_goal (coordinates,path) :
    global x_goal
    global y_goal
    global goal_reached
    if [x_goal,y_goal] == coordinates :
        goal_reached = [x_goal,y_goal] == coordinates
        set_find_path(path,coordinates)
    return goal_reached

# See if ending point is not a fall_bytes point to avoid infinit loop
def unsafe_end (x,y) :
    return [x,y] in fall_bytes

def set_find_path(root_path, coordinates) :
    global find_path
    find_path = list(root_path)
    find_path.append(list(coordinates))
    return find_path

def path_finder ():
    global paths
    global new_paths
    global steps
    #print("Goal to reach "+str(x_goal)+","+str(y_goal))
    discover_path(x_start,y_start, paths[0])
    while not(goal_reached) : 
        #print()
        #print("Next step",steps)
        paths = list(new_paths)
        new_paths = []
        steps += 1

        if paths == [] :
            print("No path found")
            return False
    
        i=0
        for path in paths :
            last_coord = len(paths[i])-1
            discover_path(paths[i][last_coord][0],paths[i][last_coord][1], paths[i])
            i += 1
    return True


x_start = 0
y_start = 0
x_goal = max_coordinate
y_goal = max_coordinate

paths = [[[0,0]]]
new_paths = []
reached_point = list(paths)
goal_reached = True
steps = 0

print("Start searching")
if unsafe_end (x_goal, y_goal) :
    print("Unsafe endcoordinates, please change goal coordinates !")
else :
    #ToDo : Make a loop to test step by step  if it's works and take coordinates when it's sop working
    i = fall_bytes_number + 1
    while i <= max_fall_bytes and path_finder() == True :
        paths = [[[0,0]]]
        new_paths = []
        reached_point = list(paths)
        steps = 0
        goal_reached = False
        i+=1
        fall_bytes = cut_fall_bytes_list(list(all_fall_bytes), i)
        
        # print()
        # print("Reach memory end with",len(find_path)-1,"steps")
        #print(find_path)
        # print()
        # print_map(find_path)
        # print()
        print("Try for "+str(i)+" bytes falls !")
    print("The falling bytes",all_fall_bytes[i-1],"after "+str(i)+" fall")


Start searching
Try for 2932 bytes falls !
Try for 2933 bytes falls !
Try for 2934 bytes falls !
Try for 2935 bytes falls !
Try for 2936 bytes falls !
Try for 2937 bytes falls !
No path found
The falling bytes [18, 62] after 2937 fall
